## 🤖 Agent Logic Overview — How the SQL Agent Works

This notebook demonstrates how to build a reflective SQL Agent using Large Language Models (LLMs). The agent is designed to take a natural-language question, generate an initial SQL query, evaluate its correctness using real execution results, and refine the query if needed. This process ensures that the final output better matches the user's intent.

---
### 🎯 Agent Objective

To create a reliable SQL Agent that:
- Understands natural-language questions
- Generates SQL queries based on schema context
- Evaluates query results to detect semantic errors
- Refines queries through iterative feedback
- Produces accurate, business-aligned answers

---

### 🧩 Agent Workflow Structure

The agent follows a multi-step workflow, inspired by modular LLM orchestration:

1. **LLM #1 — SQL Generation**  
   Receives the user’s question and schema, and generates an initial SQL query (V1).

2. **Execution Engine**  
   Runs the V1 query against the database and returns the actual output.

3. **LLM #2 — Reflection**  
   Reviews the original query and its output to identify logical or semantic issues.

4. **LLM #3 — Refinement**  
   Produces a refined SQL query (V2) that corrects any issues found.

5. **Execution Engine**  
   Runs the V2 query and returns the final answer.

> *Note: Each LLM step can use a different model with specialized capabilities (e.g., generation vs. evaluation).*

---

### 🔁 Agentic Reflection Mechanisms

- **Static Reflection**:  
  Reviews the SQL query structure without executing it. Useful for catching syntax issues, missing filters, or incorrect aggregations.

- **Dynamic Reflection**:  
  Evaluates the SQL query based on its actual output. Detects subtle semantic errors such as negative totals, empty results, or incorrect logic.

---

### 📊 Agent Logic Flow 
![SQL Agent Workflow](image.png)

---



## 🧠 Part 1 — Key Takeaways: What I Learned

### 1.1 技术能力总结

*   如何使用 LLM 将自然语言转化为 SQL 查询

*   如何构建 event-sourced 数据库并提取 schema
*   如何执行 SQL 并用 pandas 展示结果
*   如何使用反思机制修复语义错误（静态 + 动态）

### 1.2 Agentic 思维方式总结

*   为什么不能只依赖模型的第一次输出

*   如何通过执行结果发现隐藏问题
*   如何构建具备“现实感知”的 SQL Agent
*   从“模型调用”到“Agent行为”的转变


## 🔁 Part 2 — End-to-End SQL Agent Workflow

### 环境准备

In [5]:
import json
import utils
import pandas as pd
from dotenv import load_dotenv

_ = load_dotenv()

import aisuite as ai

client = ai.Client()

### 数据准备

In [6]:
utils.create_transactions_db()

utils.print_html(utils.get_schema('products.db'))

SQLite database 'products.db' created with a single 'transactions' table (event-sourced).


### 函数定义

1. generate_sql()

> 功能：根据用户提出的自然语言问题和数据库结构，生成一条 SQL 查询语句。

In [7]:
def generate_sql(question: str, schema: str, model: str) -> str:
    """
    根据用户的问题和数据库结构，生成一条 SQL 查询语句。

    参数：
    - question: 用户提出的问题（自然语言）
    - schema: 数据库的表结构（字符串形式）
    - model: 使用的语言模型名称（如 dashscope:qwen3-vl-plus）

    返回：
    - SQL 查询语句（字符串）
    """

    # 构造提示词，告诉模型它是一个 SQL 助手
    # 提供 schema 和用户问题，要求只返回 SQL 查询语句
    prompt = f"""
    You are a SQL assistant. Given the schema and the user's question, write a SQL query for SQLite.

    Schema:
    {schema}

    User question:
    {question}

    Respond with the SQL only.
    """

    # 调用语言模型生成 SQL 查询
    response = client.chat.completions.create(
        model=model,  # 指定使用的模型
        messages=[{"role": "user", "content": prompt}],  # 提示词作为用户消息传入
        temperature=0,  # 设置为 0 表示输出更稳定、确定性更强
    )

    # 提取模型返回的 SQL 内容，并去除首尾空格
    return response.choices[0].message.content.strip()


2. evaluate_sql()

> 💡 Note: The function `execute_sql()` is defined in `utils.py` for modularity and reuse. It takes a SQL string and a database path, and returns the query result as a pandas DataFrame.


3. refine_sql_external_feedback()

结合执行结果进行语义反思与修复。


In [8]:
def refine_sql_external_feedback(
    question: str,
    sql_query: str,
    df_feedback: pd.DataFrame,
    schema: str,
    model: str,
) -> tuple[str, str]:
    """
    根据 SQL 查询的执行结果进行反思，并生成修复后的 SQL 查询。

    参数：
    - question: 用户提出的问题（自然语言）
    - sql_query: 初始 SQL 查询语句（V1）
    - df_feedback: SQL 执行结果（DataFrame）
    - schema: 数据库表结构
    - model: 使用的语言模型名称

    返回：
    - feedback: 模型对初始 SQL 的评价与建议（字符串）
    - refined_sql: 修复后的 SQL 查询语句（V2）
    """

    # 构造提示词，要求模型评估 SQL 输出是否合理，并提供修复建议
    prompt = f"""
    You are a SQL reviewer and refiner.

    User asked:
    {question}

    Original SQL:
    {sql_query}

    SQL Output:
    {df_feedback.to_markdown(index=False)}

    Table Schema:
    {schema}

    Step 1: Briefly evaluate if the SQL output answers the user's question.
    Step 2: If the SQL could be improved, provide a refined SQL query.
    If the original SQL is already correct, return it unchanged.

    Return a strict JSON object with two fields:
    - "feedback": brief evaluation and suggestions
    - "refined_sql": the final SQL to run
    """

    # 调用模型进行反思与修复
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,  # 设置为 1.0，鼓励模型提出多样化修复建议
    )

    # 尝试解析模型返回的 JSON 格式内容
    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # 如果模型返回的不是有效 JSON，则保留原始 SQL 并将内容作为反馈
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql


### SQL Agent 工作流

In [21]:
def run_sql_workflow(
    db_path: str,
    question: str,
    model_generation: str = "deepseek:deepseek-chat",
    model_evaluation: str = "dashscope:qwen3-vl-plus",
    return_results: bool = True
):
    print(f"🔧 Using model: {model_generation} for generation, {model_evaluation} for evaluation")

    # Step 1: Schema
    schema = utils.get_schema(db_path)
    utils.print_html(schema, title="📘 Step 1 — Extract Database Schema")

    # Step 2: Generate SQL (V1)
    sql_v1 = generate_sql(question, schema, model_generation)
    utils.print_html(sql_v1, title="🧠 Step 2 — Generate SQL (V1)")

    # Step 3: Execute V1
    df_v1 = utils.execute_sql(sql_v1, db_path)
    utils.print_html(df_v1, title="🧪 Step 3 — Execute V1 (SQL Output)")

    # Step 4: Reflect + Refine
    feedback, sql_v2 = refine_sql_external_feedback(
        question=question,
        sql_query=sql_v1,
        df_feedback=df_v1,
        schema=schema,
        model=model_evaluation,
    )
    utils.print_html(feedback, title="🧭 Step 4 — Reflect on V1 (Feedback)")
    utils.print_html(sql_v2, title="🔁 Step 4 — Refined SQL (V2)")

    # Step 5: Execute V2
    df_v2 = utils.execute_sql(sql_v2, db_path)
    utils.print_html(df_v2, title="✅ Step 5 — Execute V2 (Final Answer)")

    if return_results:
        return {
            "sql_v1": sql_v1,
            "df_v1": df_v1,
            "feedback": feedback,
            "sql_v2": sql_v2,
            "df_v2": df_v2
        }


In [22]:
run_sql_workflow(
    "products.db", 
    "What is the total sales amount per color?",
    model_generation="deepseek:deepseek-chat",
    model_evaluation="dashscope:qwen3-vl-plus"
)

🔧 Using model: deepseek:deepseek-chat for generation, dashscope:qwen3-vl-plus for evaluation


color,total_sales_amount
black,-275176.15
blue,-190571.46
green,-214464.70
red,-242075.23
white,-358315.09


color,total_sales_amount
black,275176.15
blue,190571.46
green,214464.70
red,242075.23
white,358315.09


{'sql_v1': "```sql\nSELECT \n    color,\n    SUM(qty_delta * unit_price) AS total_sales_amount\nFROM transactions\nWHERE action = 'sale' AND qty_delta < 0\nGROUP BY color;\n```",
 'df_v1':    color  total_sales_amount
 0  black          -275176.15
 1   blue          -190571.46
 2  green          -214464.70
 3    red          -242075.23
 4  white          -358315.09,
 'feedback': 'The SQL correctly calculates total sales amount per color, but the negative values may be misleading since sales amounts are typically expressed as positive. The query should multiply by -1 to reflect actual revenue.',
 'sql_v2': "SELECT \n    color,\n    SUM(qty_delta * unit_price * -1) AS total_sales_amount\nFROM transactions\nWHERE action = 'sale' AND qty_delta < 0\nGROUP BY color;",
 'df_v2':    color  total_sales_amount
 0  black           275176.15
 1   blue           190571.46
 2  green           214464.70
 3    red           242075.23
 4  white           358315.09}

### 模型对比实验

In [24]:
def compare_model_combinations(
    db_path: str,
    question: str,
    model_pairs: list[tuple[str, str]]
) -> list[dict]:
    """
    对比不同模型组合在 SQL Agent 工作流中的表现。

    参数：
    - db_path: 数据库路径
    - question: 用户问题
    - model_pairs: 模型组合列表，如 [("deepseek-chat", "qwen3-vl-plus"), ...]

    返回：
    - 每组模型的执行结果列表（字典形式）
    """
    results = []

    for gen_model, eval_model in model_pairs:
        print(f"\n🔍 Testing: Gen={gen_model}, Eval={eval_model}")

        output = run_sql_workflow(
            db_path=db_path,
            question=question,
            model_generation=gen_model,
            model_evaluation=eval_model,
            return_results=True
        )

        # 判断是否发生语义修复
        repaired = output["sql_v1"].strip() != output["sql_v2"].strip()

        # 判断最终结果是否为正值（逻辑正确）
        df = output["df_v2"]
        valid = df.select_dtypes(include="number").gt(0).all().all()

        results.append({
            "gen_model": gen_model,
            "eval_model": eval_model,
            "sql_v1": output["sql_v1"],
            "sql_v2": output["sql_v2"],
            "repaired": repaired,
            "valid_result": valid,
            "feedback": output["feedback"],
            "df_v2": df
        })

    return results


In [26]:
model_pairs = [
    ("deepseek:deepseek-chat", "dashscope:qwen3-vl-plus"),
    ("dashscope:qwen3-vl-plus", "deepseek:deepseek-chat"),
    ("dashscope:qwen3-vl-plus", "dashscope:qwen3-vl-plus"),
    ("deepseek:deepseek-chat", "deepseek:deepseek-chat"),
]

results = compare_model_combinations(
    db_path="products.db",
    question="What is the total sales amount per color?",
    model_pairs=model_pairs
)



🔍 Testing: Gen=deepseek:deepseek-chat, Eval=dashscope:qwen3-vl-plus
🔧 Using model: deepseek:deepseek-chat for generation, dashscope:qwen3-vl-plus for evaluation


color,total_sales_amount
black,-275176.15
blue,-190571.46
green,-214464.70
red,-242075.23
white,-358315.09


color,total_sales_amount



🔍 Testing: Gen=dashscope:qwen3-vl-plus, Eval=deepseek:deepseek-chat
🔧 Using model: dashscope:qwen3-vl-plus for generation, deepseek:deepseek-chat for evaluation


color,total_sales_amount
black,-275176.15
blue,-190571.46
green,-214464.70
red,-242075.23
white,-358315.09


...

color,total_sales_amount
black,-275176.15
blue,-190571.46
green,-214464.70
red,-242075.23
white,-358315.09


color,total_sales_amount
black,-275176.15
blue,-190571.46
green,-214464.70
red,-242075.23
white,-358315.09



🔍 Testing: Gen=deepseek:deepseek-chat, Eval=deepseek:deepseek-chat
🔧 Using model: deepseek:deepseek-chat for generation, deepseek:deepseek-chat for evaluation


color,total_sales_amount
black,-275176.15
blue,-190571.46
green,-214464.70
red,-242075.23
white,-358315.09


color,total_sales_amount
black,-275176.15
blue,-190571.46
green,-214464.70
red,-242075.23
white,-358315.09


In [27]:
pd.DataFrame([{
    "Gen": r["gen_model"],
    "Eval": r["eval_model"],
    "Repaired": r["repaired"],
    "Valid": r["valid_result"]
} for r in results])

,Gen,Eval,Repaired,Valid
0,deepseek:deepseek-chat,dashscope:qwen3-vl-plus,True,True
1,dashscope:qwen3-vl-plus,deepseek:deepseek-chat,True,True
2,dashscope:qwen3-vl-plus,dashscope:qwen3-vl-plus,False,False
3,deepseek:deepseek-chat,deepseek:deepseek-chat,False,False


#### 🧠 Model Comparison Summary

This experiment tested four combinations of SQL generation and evaluation models:

| Generation Model | Evaluation Model | Repaired | Valid |
|------------------|------------------|----------|-------|
| DeepSeek         | Qwen             | ✅        | ✅     |
| Qwen             | DeepSeek         | ✅        | ✅     |
| Qwen             | Qwen             | ❌        | ❌     |
| DeepSeek         | DeepSeek         | ❌        | ❌     |

> 💡 Insight: Using different models for generation and evaluation significantly improves semantic accuracy. Cross-model reflection enables the agent to detect and correct logical errors that single-model workflows miss.


## ✅ Part 3 Summary — End-to-End SQL Agent & Model Comparison

We built and tested a modular SQL Agent workflow with the following capabilities:

### 🔁 Workflow Steps
1. Extract database schema
2. Generate SQL (V1) using a language model
3. Execute SQL and display output
4. Reflect on SQL output and propose refined SQL (V2)
5. Execute V2 and display final answer

### 🧠 Modular Design
- All functions are defined in `utils.py` or notebook cells
- Supports flexible model selection for generation and evaluation
- Results are displayed inline using `print_html()`

---

### 🧪 Model Comparison Results

We tested four combinations of generation and evaluation models:

| Generation Model        | Evaluation Model        | Repaired | Valid |
|-------------------------|-------------------------|----------|-------|
| `deepseek:deepseek-chat` | `dashscope:qwen3-vl-plus` | ✅        | ✅     |
| `dashscope:qwen3-vl-plus` | `deepseek:deepseek-chat` | ✅        | ✅     |
| `dashscope:qwen3-vl-plus` | `dashscope:qwen3-vl-plus` | ❌        | ❌     |
| `deepseek:deepseek-chat` | `deepseek:deepseek-chat` | ❌        | ❌     |

> 💡 Insight: Using different models for generation and evaluation significantly improves semantic accuracy. Cross-model reflection enables the agent to detect and correct logical errors that single-model workflows miss.

---

## 📦 Next Steps (Optional Extensions)

- Add chart visualization to compare V1 vs V2 results
- Support user-uploaded databases for custom queries
- Build a reusable knowledge base summarizing schema, queries, and model 